In [ ]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy import interpolate, integrate
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

import seaborn as sns
from glob import glob
import sys
sys.path.insert(1, '../')
import emcee

%matplotlib inline

plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')
plt.rc('font', size=15)
plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=16, labelsize=17)
plt.rc('lines', linewidth=2, markersize=6)
plt.rc('legend', fontsize=15)

Results plots
=========

In [ ]:
n_disc = 300
l_max_array = [99, 79, 49, 24, 15]

filenames = glob('../Data/chains/No_Om_prior/*.h5')
N = len(filenames)
print filenames

SNr = np.zeros(N)
K0_bsqr_No_Om_prior, z_p_No_Om_prior, Om_No_Om_prior \
    = np.zeros(N), np.zeros(N), np.zeros(N)

for i, filename in enumerate(filenames):
    backend = emcee.backends.HDFBackend(filename, read_only=True)
    samps = backend.get_chain(flat = True, discard=n_disc)
    
    fac = int(filename.strip('../Data/chains/No_Om_prior/chain_')[0:1])
    
    print fac, filename
    K0_bsqr_No_Om_prior[i] = (np.percentile(samps[:, 1], 50.+34.1) - np.percentile(samps[:, 1], 50.-34.1))/2.
    z_p_No_Om_prior[i] = (np.percentile(samps[:, 2], 50.+34.1) - np.percentile(samps[:, 2], 50.-34.1))/2.
    Om_No_Om_prior[i] = (np.percentile(samps[:, 3], 50.+34.1) - np.percentile(samps[:, 3], 50.-34.1))/2.
    
    l_max = l_max_array[fac - 1]
    SNr[i] = np.sqrt(3.*((2*np.arange(10, l_max + 1) + 1.)/2.).sum())
    
    
filenames = glob('../Data/chains/Om_prior/*.h5')
N = len(filenames)
print filenames
K0_bsqr_Om_prior, z_p_Om_prior, Om_Om_prior \
    = np.zeros(N), np.zeros(N), np.zeros(N)

for i, filename in enumerate(filenames):
    backend = emcee.backends.HDFBackend(filename, read_only=True)
    samps = backend.get_chain(flat = True, discard=n_disc)
    
    fac = int(filename.strip('../Data/chains/Om_prior/chain_')[0:1])
    
    print fac, filename
    K0_bsqr_Om_prior[i] = (np.percentile(samps[:, 1], 50.+34.1) - np.percentile(samps[:, 1], 50.-34.1))/2.
    z_p_Om_prior[i] = (np.percentile(samps[:, 2], 50.+34.1) - np.percentile(samps[:, 2], 50.-34.1))/2.
    Om_Om_prior[i] = (np.percentile(samps[:, 3], 50.+34.1) - np.percentile(samps[:, 3], 50.-34.1))/2.
    
    

In [ ]:
fig = plt.figure()
ax = plt.gca()


idxs = np.argsort(SNr)
print idxs

snr_lst = SNr[idxs]
[ax.axvline(x = snr_tmp, alpha = 0.2, ls = "-") for snr_tmp in snr_lst]

z_p_lst = z_p_No_Om_prior[idxs]
Om_lst = Om_No_Om_prior[idxs]
K0_bsqr_lst = K0_bsqr_No_Om_prior[idxs]

ax.plot(snr_lst, Om_lst/0.32, ls='--', c = 'gray', alpha = 0.5)
ax.plot(snr_lst, K0_bsqr_lst, ls='--', c = 'gray', alpha = 0.5)
ax.plot(snr_lst, z_p_lst, ls='--', c = 'gray', alpha = 0.5)

col = "darkorange"#sns.color_palette("Purples", 8)[-1]

ax.scatter(snr_lst, Om_lst/0.32, label=r'$\Omega_m$', marker = "s", c = col, alpha = 1.)
ax.scatter(snr_lst, K0_bsqr_lst, label=r'$b_\mathrm{GW}^2 \mathcal{K}_0$', c = col, alpha = 1.0)
ax.scatter(snr_lst, z_p_lst, label=r'$z_\ast$', marker = "*", c = col, alpha = 1.0)


#ax.set_xlim([10, 99])
ax.set_yscale("log")
ax.set_ylim([5e-3, 5e-1])

ax1 = ax.twiny()


ax1.spines["top"].set_position(("axes", 1.))
ax1.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])

plt.xticks(snr_lst, l_max_array[::-1])

ax1.set_xlabel(r'$\ell_\mathrm{max}$')



#ax.yaxis.set_major_locator(MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.05))

ax.set_ylabel(r'$\sigma_\theta/\theta_\mathrm{fid}$')
ax.set_xlabel(r'$\mathrm{S}/\mathrm{N}$')



ax.set_title('Flat prior on ' + r'$\Omega_\mathrm{M}$', pad = 50)
ax.legend(loc = 'upper right')

ax.grid(True, which = 'minor', alpha = 0.2, ls = "-")

fig.tight_layout()
plt.savefig("Figs/main_result_flat_prior.pdf")

plt.show()

In [ ]:
fig = plt.figure()
ax = plt.gca()


idxs = np.argsort(SNr)
print idxs

snr_lst = SNr[idxs]
[ax.axvline(x = snr_tmp, alpha = 0.2, ls = "-") for snr_tmp in snr_lst]



z_p_lst = z_p_Om_prior[idxs]
Om_lst = Om_Om_prior[idxs]
K0_bsqr_lst = K0_bsqr_Om_prior[idxs]

ax.plot(snr_lst, Om_lst/0.32, ls='--', c = 'gray', alpha = 1.)
ax.plot(snr_lst, K0_bsqr_lst, ls='--', c = 'gray', alpha = 1.)
ax.plot(snr_lst, z_p_lst, ls='--', c = 'gray', alpha = 1.)

col = 'black'#sns.color_palette("YlOrRd", 5)[-2]

ax.scatter(snr_lst, Om_lst/0.32, label=r'$\Omega_m$', marker = "s", c = col)
ax.scatter(snr_lst, K0_bsqr_lst, label=r'$b_\mathrm{GW}^2 \mathcal{K}_0$', c = col)
ax.scatter(snr_lst, z_p_lst, label=r'$z_\ast$', marker = "*", c = col)



ax1 = ax.twiny()


ax1.spines["top"].set_position(("axes", 1.))
ax1.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])

plt.xticks(snr_lst, l_max_array[::-1])
ax1.set_xlabel(r'$\ell_\mathrm{max}$')



#ax.set_xlim([10, 99])
ax.set_ylim([5e-3, 2e-1])
ax.set_yscale("log")


#ax.yaxis.set_major_locator(MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.05))

ax.set_ylabel(r'$\sigma_\theta/\theta_\mathrm{fid}$')
ax.set_xlabel(r'$\mathrm{S}/\mathrm{N}$')



ax.set_title('Gaussian prior on ' + r'$\Omega_\mathrm{M}$', pad = 50)
ax.legend(loc = 'upper right')

ax.grid(True, which = 'minor', alpha = 0.2, ls = "-")

fig.tight_layout()
plt.savefig("Figs/main_result_gaussian_prior.pdf")

plt.show()